In [ ]:
# AI Concept Engine for Architecture Students

#Generates 50+ concept directions from a single prompt:
#- Diagrams & massing ideas (text-based + prompt-ready)
#- Narrative hooks & phenomenological frames
#- Moodboard prompts & precedent suggestions
#- Anti–blank page modes (wild/safe/studio-friendly/etc.)


In [ ]:
# @title Install & imports
!pip -q install openai==1.6.1

import random
import textwrap
from dataclasses import dataclass, asdict
from typing import List, Dict, Any

from openai import OpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 10.6 MB/s eta 0:00:00


In [ ]:
# @title 🔑 API key & client setup
# @markdown Enter your OpenAI (or compatible) API key here.

API_KEY = "YOUR_API_KEY_HERE"  # @param {type:"string"}

if not API_KEY or API_KEY == "YOUR_API_KEY_HERE":
    raise ValueError("Please set your API key in the API_KEY variable.")

client = OpenAI(api_key=API_KEY)
MODEL_NAME = "gpt-4.1-mini"  # You can change this

ValueError: Please set your API key in the API_KEY variable.

In [ ]:
# @title ⚙️ Global configuration

NUM_CONCEPTS = 50  # @param {type:"integer"}
MAX_TOKENS_PER_CALL = 1800  # safety limit

# Style controls
CONCEPT_STYLE = "studio-crit, architectural, concise but evocative"
NARRATIVE_TONE = "poetic, phenomenological, but still clear"
MOODBOARD_STYLE = "visual, cinematic, material-focused"
PRECEDENT_STYLE = "architecture-school, global, 20th–21st century"

WRAP_WIDTH = 90

In [ ]:
# @title 🧱 Data structures

@dataclass
class Concept:
    id: int
    title: str
    thesis: str
    diagram: str
    massing: str
    narrative: str
    moodboard_prompts: List[str]
    precedents: List[Dict[str, str]]  # {name, type, why}


def wrap(text: str, width: int = WRAP_WIDTH) -> str:
    return "\n".join(textwrap.wrap(text, width=width))

In [ ]:
# @title 🧠 LLM helper

def call_llm(system_prompt: str, user_prompt: str, model: str = MODEL_NAME) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=MAX_TOKENS_PER_CALL,
        temperature=0.9,
        top_p=0.95,
    )
    return response.choices[0].message.content.strip()

NameError: name 'MODEL_NAME' is not defined

In [ ]:
# @title 📝 Project prompt input
# @markdown Describe your project below. Be as detailed or as loose as you like.

site_description = "Urban corner site in London, adjacent to a small park."  # @param {type:"string"}
program_description = "Mixed-use building with ground floor public space, studios, and housing above."  # @param {type:"string"}
constraints = "Mid-rise height limit, strong street edge, noise from main road, good southern light."  # @param {type:"string"}
keywords = "porosity, thresholds, light wells, courtyards"  # @param {type:"string"}
vibe_materiality = "soft brutalism, warm concrete, timber interiors, planted terraces"  # @param {type:"string"}

project_brief = f"""
Site: {site_description}
Program: {program_description}
Constraints: {constraints}
Keywords: {keywords}
Vibe & Materiality: {vibe_materiality}
""".strip()

print("📌 Project brief:\n")
print(wrap(project_brief))

In [ ]:
# @title 🧩 Concept generator prompt template

def build_concept_generation_prompt(brief: str, num_concepts: int) -> str:
    return f"""
You are an experienced architecture studio critic.

Given the following project brief:

{brief}

Generate exactly {num_concepts} distinct architectural concept directions.

For EACH concept, output in this STRICT JSON-like structure (no extra commentary):

[
  {{
    "id": 1,
    "title": "Short concept title",
    "thesis": "One-sentence conceptual thesis.",
    "diagram": "Describe a simple diagram (plan/section/axon).",
    "massing": "Describe a massing strategy.",
    "narrative": "Short narrative hook explaining why this matters."
  }},
  ...
]

Rules:
- Concepts must be spatially and diagrammatically distinct.
- Use clear, studio-crit language.
- Avoid repetition of titles and ideas.
- Do NOT include moodboards or precedents here.
- Output ONLY the JSON array, no prose around it.
"""

In [ ]:
# @title 🚀 Generate raw concepts (titles, thesis, diagram, massing, narrative)

raw_concepts_text = call_llm(
    system_prompt=f"You generate architectural concept directions in a {CONCEPT_STYLE} style.",
    user_prompt=build_concept_generation_prompt(project_brief, NUM_CONCEPTS),
)

print(raw_concepts_text[:2000] + ("\n...\n[truncated preview]" if len(raw_concepts_text) > 2000 else ""))

NameError: name 'call_llm' is not defined

In [ ]:
# @title 🧪 Parse concepts into Python objects

import json
import re

def safe_json_parse(text: str) -> Any:
    # Try to extract JSON array heuristically
    match = re.search(r"\[.*\]", text, re.DOTALL)
    if not match:
        raise ValueError("Could not find JSON array in LLM output.")
    json_str = match.group(0)
    return json.loads(json_str)

parsed = safe_json_parse(raw_concepts_text)

concepts: List[Concept] = []
for item in parsed:
    c = Concept(
        id=int(item.get("id", len(concepts) + 1)),
        title=item.get("title", "").strip(),
        thesis=item.get("thesis", "").strip(),
        diagram=item.get("diagram", "").strip(),
        massing=item.get("massing", "").strip(),
        narrative=item.get("narrative", "").strip(),
        moodboard_prompts=[],
        precedents=[],
    )
    concepts.append(c)

print(f"Parsed {len(concepts)} concepts.")
for c in concepts[:3]:
    print(f"- [{c.id}] {c.title}")

NameError: name 'raw_concepts_text' is not defined

In [ ]:
# @title 🎨 Moodboard & precedent generator prompt

def build_moodboard_precedent_prompt(brief: str, concept: Concept) -> str:
    return f"""
You are an architecture tutor and visual researcher.

Project brief:
{brief}

Concept:
ID: {concept.id}
Title: {concept.title}
Thesis: {concept.thesis}
Diagram: {concept.diagram}
Massing: {concept.massing}
Narrative: {concept.narrative}

TASK 1 — Moodboard prompts:
Generate 8–12 short, text-only prompts suitable for image generation tools.
Each should describe:
- Atmosphere
- Material palette
- Light & shadow
- Key spatial gesture

TASK 2 — Precedent suggestions:
Suggest 3–5 architectural precedents and 1–2 non-architectural references (film, art, landscape, biology, etc.).
For each, explain briefly WHY it aligns with the concept.

Output in this STRICT JSON structure:

{{
  "moodboard_prompts": [
    "prompt 1",
    "prompt 2",
    ...
  ],
  "precedents": [
    {{
      "name": "Precedent name",
      "type": "architecture / film / art / landscape / biology / etc.",
      "why": "Short explanation."
    }},
    ...
  ]
}}

Rules:
- Use a {MOODBOARD_STYLE} style for moodboards.
- Use a {PRECEDENT_STYLE} style for precedents.
- Output ONLY the JSON object, no extra commentary.
"""

In [ ]:
# @title 🔁 Enrich all concepts with moodboards & precedents

def enrich_concept(concept: Concept, brief: str) -> Concept:
    text = call_llm(
        system_prompt="You generate moodboard prompts and precedents for architecture students.",
        user_prompt=build_moodboard_precedent_prompt(brief, concept),
    )
    try:
        data = safe_json_parse(text.replace("\n", " "))
    except Exception as e:
        print(f"Parsing error for concept {concept.id} - {concept.title}: {e}")
        return concept

    concept.moodboard_prompts = data.get("moodboard_prompts", [])
    concept.precedents = data.get("precedents", [])
    return concept

for i, c in enumerate(concepts, start=1):
    print(f"Enriching concept {i}/{len(concepts)}: {c.title}")
    concepts[i-1] = enrich_concept(c, project_brief)

print("Done.")

NameError: name 'concepts' is not defined

In [ ]:
# @title 🎲 Anti–blank page filters

def filter_concepts_by_mode(concepts: List[Concept], mode: str, k: int = 10) -> List[Concept]:
    """
    Simple heuristic: we classify by keywords in title/thesis/narrative.
    You can refine this later.
    """
    text_map = {
        "wild": ["fragment", "drift", "fold", "void", "hybrid", "mutant", "field"],
        "safe": ["courtyard", "bar", "tower", "block", "terrace", "atrium"],
        "studio": ["section", "threshold", "porous", "gradient", "edge", "spine"],
        "diagram": ["grid", "axis", "loop", "cross", "hinge", "stack"],
        "narrative": ["memory", "ritual", "story", "journey", "sequence", "choreography"],
    }

    keywords = text_map.get(mode, [])
    scored = []
    for c in concepts:
        text = f"{c.title} {c.thesis} {c.narrative}".lower()
        score = sum(1 for kw in keywords if kw in text)
        scored.append((score, c))
    scored.sort(key=lambda x: x[0], reverse=True)
    return [c for score, c in scored[:k]]

mode = "wild"  # @param ["wild", "safe", "studio", "diagram", "narrative"]
subset = filter_concepts_by_mode(concepts, mode, k=10)

print(f"Mode: {mode} — showing {len(subset)} concepts\n")
for c in subset:
    print(f"[{c.id}] {c.title} — {c.thesis}")

NameError: name 'concepts' is not defined

In [ ]:
# @title 📄 View a single concept sheet

concept_id = 1  # @param {type:"integer"}

def get_concept_by_id(concepts: List[Concept], cid: int) -> Concept:
    for c in concepts:
        if c.id == cid:
            return c
    raise ValueError(f"No concept with id {cid}")

c = get_concept_by_id(concepts, concept_id)

print(f"=== Concept {c.id}: {c.title} ===\n")
print("Thesis:")
print(wrap(c.thesis), "\n")

print("Diagram idea:")
print(wrap(c.diagram), "\n")

print("Massing idea:")
print(wrap(c.massing), "\n")

print("Narrative hook:")
print(wrap(c.narrative), "\n")

print("Moodboard prompts:")
for i, m in enumerate(c.moodboard_prompts, start=1):
    print(f"  {i:02d}. {wrap(m)}")

print("\nPrecedents:")
for p in c.precedents:
    print(f"- {p.get('name', 'Unknown')} ({p.get('type', 'n/a')}): {wrap(p.get('why', ''))}")

NameError: name 'concepts' is not defined

In [ ]:
# @title 📌 Export: pin-up style text block (copy/paste)

def concept_to_pinup_block(c: Concept) -> str:
    lines = []
    lines.append(f"CONCEPT {c.id}: {c.title}")
    lines.append("-" * 60)
    lines.append("THESIS:")
    lines.append(c.thesis)
    lines.append("")
    lines.append("DIAGRAM:")
    lines.append(c.diagram)
    lines.append("")
    lines.append("MASSING:")
    lines.append(c.massing)
    lines.append("")
    lines.append("NARRATIVE:")
    lines.append(c.narrative)
    lines.append("")
    lines.append("MOODBOARD PROMPTS:")
    for i, m in enumerate(c.moodboard_prompts, start=1):
        lines.append(f"{i:02d}. {m}")
    lines.append("")
    lines.append("PRECEDENTS:")
    for p in c.precedents:
        lines.append(f"- {p.get('name', 'Unknown')} ({p.get('type', 'n/a')}): {p.get('why', '')}")
    return "\n".join(lines)

concept_ids_to_export = [c.id for c in concepts[:5]]  # @param {type:"raw"}

blocks = []
for cid in concept_ids_to_export:
    c = get_concept_by_id(concepts, cid)
    blocks.append(concept_to_pinup_block(c))

export_text = "\n\n" + ("=" * 80) + "\n\n"
export_text = export_text.join(blocks)

print(export_text)

NameError: name 'concepts' is not defined

In [ ]:
# @title 📦 Export all concepts as JSON

all_concepts_json = [asdict(c) for c in concepts]

import json
json_str = json.dumps(all_concepts_json, indent=2, ensure_ascii=False)

print(json_str[:3000] + ("\n...\n[truncated preview]" if len(json_str) > 3000 else ""))

NameError: name 'concepts' is not defined